In [ ]:
import math

import pandas as pd
from ipywidgets import widgets
from bokeh.io import output_notebook, push_notebook
from bokeh.plotting import figure, ColumnDataSource, show
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.layouts import gridplot

output_notebook()

In [ ]:
def create_sin_wave(amplitude=1, period=(2*math.pi), phase=0):
    """Create x and y variables for a sin wave (-2pi to 2pi).
    
    Arguments:
        fn (str): Either 'sin' or 'cos'.
        amplitude (float): The amplitude of the wave; default = 1.
        period (float): The period of the wave; default = 2pi.
        phase (float): The phase of the wave; default = 0.
    
    Returns:
        ([float], [float]): The x and y coordinates of the wave.
    """
    x = [2 * (i - 100) * math.pi / 100 for i in range(200)]
    sin = [amplitude * math.sin((2 * math.pi * i) / period + phase) for i in x]
    cos = [amplitude * math.cos((2 * math.pi * i) / period + phase) for i in x]
    return pd.DataFrame(list(zip(x, sin, cos)), columns=['x', 'sin', 'cos'])

## Review

From the previous lecture on Jupyter, to plot something, we first create a DataFrame of the values. Here we just have a sine wave:

In [ ]:
df0 = create_sin_wave()
df0.head()

Then we can plot it using bokeh:

In [ ]:
# create and show the chart
fig0 = figure(width=600, height=300)
fig0.circle(
    x='x',
    y='sin',
    size=5,
    source=ColumnDataSource(df0),
)
show(fig0)

## Hover

[Tutorial/Documentation](https://bokeh.pydata.org/en/latest/docs/user_guide/tools.html#hovertool)

Bokeh supports a number of interactions at the chart level, called *tools*. One of the most basic (beyond panning, etc.) is to display information on hover. Note the two modifications to the code:

* The return value of `f.circle` is saved into the `renderer` variable on line 5.
* We call `add_tools()` on the figure on lines 12-17. The tool we add is `HoverTool`, whose constructor we call with two keyword arguments:
    * `renderers`, which takes the saved `renderer` in a list
    * `tooltips`, which takes a list of tuples to display. The first element in the tuple is the label; the second element is the value. Bokeh supports a syntax where `@x` means "look up the `"x"` column the data", so the value for hovering over each point is different.

In [ ]:
df1 = create_sin_wave()

# create and show the chart
fig1 = figure(width=600, height=300)
renderer1 = fig1.circle(
    x='x',
    y='sin',
    size=5,
    source=ColumnDataSource(df1),
)

# add hovertool to the chart
fig1.add_tools(HoverTool(renderers=[renderer1], tooltips=[
    ('Function', 'sin'),
    ('x', '@x'),
    ('y', '@sin'),
]))

show(fig1)

## Multiple Plots

[Tutorials/Documentation](https://bokeh.pydata.org/en/latest/docs/user_guide/layout.html)

Bokeh also supports laying out multiple plots at once. To make multiple plots, we first create multiple figures, then `show()` them as a `gridplot()` in lines 22-24. `gridplot()` takes a single nested list (ie. a 2D array) of all the figures you want to plot. In this case, we want to put the sine and cosine figures side by side, so the outer list only has one element.

In [ ]:
df2 = create_sin_wave()

# create the sin chart
sin_fig2 = figure(width=400, height=200)
sin_fig2.circle(
    x='x',
    y='sin',
    size=3,
    source=ColumnDataSource(df2),
)

# create the cos chart
cos_fig2 = figure(width=400, height=200)
cos_fig2.circle(
    x='x',
    y='cos',
    size=3,
    source=ColumnDataSource(df2),
)

# show both side by side
show(gridplot([
    [sin_fig2, cos_fig2],
]))

Just as a demo, we can plot four charts together as well:

In [ ]:
df3 = create_sin_wave()

# create the all the figures
sin_fig3_1 = figure(width=400, height=200)
sin_fig3_1.circle(
    x='x',
    y='sin',
    size=3,
    source=ColumnDataSource(df3),
)
cos_fig3_1 = figure(width=400, height=200)
cos_fig3_1.circle(
    x='x',
    y='cos',
    size=3,
    source=ColumnDataSource(df3),
)
cos_fig3_2 = figure(width=400, height=200)
cos_fig3_2.circle(
    x='x',
    y='cos',
    size=3,
    color='red',
    source=ColumnDataSource(df3),
)
sin_fig3_2 = figure(width=400, height=200)
sin_fig3_2.circle(
    x='x',
    y='sin',
    size=3,
    color='red',
    source=ColumnDataSource(df3),
)

# show the figures in a 2x2 grid
show(gridplot([
    [sin_fig3_1, cos_fig3_1],
    [cos_fig3_2, sin_fig3_2],
]))

## Linked Plots

[Tutorials/Documentation](https://bokeh.pydata.org/en/latest/docs/user_guide/interaction/linking.html)

If you use the pan tool to move individual figures, you will notice that the figures move independently. We can *link* the ranges of the figures so they move in sync. To do this, simply specify the `x_range` and `y_range` of the second figure to be the same as the first figure, as we do in line 16 below. Try panning on the resulting figures; both plots should move in sync.

In [ ]:
df4 = create_sin_wave()

# create the sin chart
sin_fig4 = figure(width=400, height=200)
renderer4 = sin_fig4.circle(
    x='x',
    y='sin',
    size=5,
    source=ColumnDataSource(df4),
)

# create the cos chart
cos_fig4 = figure(
    width=400, height=200,
    # use the range of the previous figure for this one
    x_range=sin_fig4.x_range, y_range=sin_fig4.y_range,
)
renderer4 = cos_fig4.circle(
    x='x',
    y='cos',
    size=5,
    source=ColumnDataSource(df4),
)

# show both side by side
show(gridplot([[sin_fig4, cos_fig4]]))

## Linked Selection

[Tutorial/Documentation](https://bokeh.pydata.org/en/latest/docs/user_guide/interaction/linking.html#linked-brushing)

The other thing you might want to do with multiple plots is link the selection of data, so that highlighting data on one figure also highlights the same data on the other. To pandas/bokeh, what you're actually doing is selecting *rows* of the DataFrame, then changing attributes (eg. color) of the data from the selected rows.

To do this, the source of the data for both plots must be the same. Previously, we have been creating new `ColumnDataSource`s for each figure. Here we must create it first (line 3), then use that same `ColumnDataSource` for both figures (lines 11 and 26). Finally, we need to add a `BoxSelectTool` to both figures, which (like the `HoverTool`) takes a list of renderers. We show both figures using `gridplot()` as before.

Try selecting data in one figure in the plot below:

In [ ]:
# the source for both graphs must be the same
df5 = create_sin_wave()
source5 = ColumnDataSource(df5)

# create the sin chart
sin_fig5 = figure(width=400, height=200)
renderer5 = sin_fig5.circle(
    x='x',
    y='sin',
    size=5,
    source=source5,
)
# add a BoxSelectTool to the figure
sin_fig5.add_tools(BoxSelectTool(renderers=[renderer5]))

# create the cos chart
cos_fig5 = figure(
    width=400, height=200,
    # use the range of the previous figure for this one
    x_range=sin_fig5.x_range, y_range=sin_fig5.y_range,
)
renderer5 = cos_fig5.circle(
    x='x',
    y='cos',
    size=5,
    source=source5,
)
# add a BoxSelectTool to the figure
cos_fig5.add_tools(BoxSelectTool(renderers=[renderer5]))

# show both side by side
show(gridplot([[sin_fig5, cos_fig5]]))

## Interactive Widgets

Tutorial/Documentation:
* [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html)
* [with Bokeh](https://bokeh.pydata.org/en/latest/docs/user_guide/notebook.html#jupyter-interactors)

Finally, we can tackle changing the data itself through slides, radio boxes, etc - through *widgets*. We use the `ipywidgets` library for this, but before we get there, we need to change how we represent our data. Specifically, we need a global variable - here `state` - that describes the data we are currently displaying. For ease of use, we will also create a new `create_sin_wave_from_state()` function that takes that variable as the sole argument.

In [ ]:
state = {
    'amplitude': 1,
    'period': 2 * math.pi,
    'phase': 0,
}

def create_sin_wave_from_state(state):
    """Create x and y variables for a sin wave (-2pi to 2pi).
    
    Arguments:
        state (dict): A dictionary of 'fn', 'amplitude', 'period', and 'phase' values
    
    Returns:
        ([float], [float]): The x and y coordinates of the wave.
    """
    x = [2 * (i - 100) * math.pi / 100 for i in range(200)]
    amplitude = state['amplitude']
    period = state['period']
    phase = state['phase']
    sin = [amplitude * math.sin((2 * math.pi * i) / period + phase) for i in x]
    cos = [amplitude * math.cos((2 * math.pi * i) / period + phase) for i in x]
    return pd.DataFrame(list(zip(x, sin, cos)), columns=['x', 'sin', 'cos'])

Displaying the plot is done almost exactly the same way, except that we need to pass in a `notebook_handle=True` argument to `show()` in line 18. This tells Bokeh that you will be modifying that figure later. Because we will be using this code again later, we will wrap it in a function:

Now let's create a control that will allow the reader to toggle between sine and cosine. Breaking down this code:

* Lines 1-18 is the same plotting code we have been using. Note that we are saving the `handle` on line 18 to be used in line 25, so that we can have multiple interactive components at once.
* Lines 20-25 is the `update_plot()` function, which recalculates the data from the state and updates the `source`. The `push_notebook()` function tells jupyter/bokeh to actually update the figure.
* Lines 27-30 is the `amplitude_change_hander()` function, which updates the `state` global variable with the value from the slider. The slider value comes in a `change` object; see the documentation for more details.
* Lines 32-39 creates the slider. The `value` keyword argument is the initial value of the slider; here, we read use the current value of the `state` as the initial value.
* Lines 41-42 links the slider with the `amplitude_change_hander()` function; every time the slider is moved, that function will be called. The `names` keyword argument means we only care about the value changing (as opposed to the description changing, etc.)
* Lines 44-45 displays the slider in the notebook.

All this work gives us the plot below, where we can control the amplitude of the sine wave.

In [ ]:
# create the data source
df6 = create_sin_wave()
source6 = ColumnDataSource(df6)

# create and show the chart
fig6 = figure(
    width=600,
    height=300,
    x_range=[-6, 6],
    y_range=[-2 * math.pi, 2 * math.pi],
)
fig6.circle(
    x='x',
    y='sin',
    size=5,
    source=source6,
)
handle6 = show(fig6, notebook_handle=True)

# update plot function
def update_plot6():
    new_data = create_sin_wave_from_state(state)
    source6.data['sin'] = list(new_data['sin'])
    source6.data['cos'] = list(new_data['cos'])
    push_notebook(handle=handle6)

# handler function
def amplitude_change_handler6(change):
    state['amplitude'] = change.new
    update_plot6()

# create the slider
amplitude_slider6 = widgets.FloatSlider(
    description="amplitude",
    min=0,
    max=5,
    value=state['amplitude'],
    step=.1,
)

# link the slider with the handler function
amplitude_slider6.observe(amplitude_change_handler6, names='value')

# display the slider
display(amplitude_slider6)

## Putting it All Together

Putting all of this together, we can create a small interactive app that shows a circle and a sine wave, see which points of the sine wave correspond to points of the circle, and allow the reader to change the amplitude/radius, period, and phase.

In [ ]:
# use a new global variable

final_example_state = {
    'amplitude': 1,
    'period': 2 * math.pi,
    'phase': 0,
}

# create the data source

df7 = create_sin_wave_from_state(final_example_state)
source7 = ColumnDataSource(df7)

# create the sine figure

sin_fig7 = figure(
    width=600, height=300,
    x_range=[-2 * math.pi, 2 * math.pi],
    y_range=[-6, 6],
)
sin_renderer7 = sin_fig7.circle(
    x='x',
    y='sin',
    size=5,
    source=source7,
)
sin_fig7.add_tools(
    HoverTool(renderers=[sin_renderer7], tooltips=[
        ('x', '@x'),
        ('y', '@sin'),
    ]),
    BoxSelectTool(renderers=[sin_renderer7]),
)

# create the circle figure

circ_fig7 = figure(
    width=300, height=300,
    x_range=[-5.5, 5.5], y_range=[-5.5, 5.5],
)
circ_renderer7 = circ_fig7.circle(
    x='sin',
    y='cos',
    size=5,
    source=source7,
)
circ_fig7.add_tools(
    HoverTool(renderers=[circ_renderer7], tooltips=[
        ('x', '@sin'),
        ('y', '@cos'),
    ]),
    BoxSelectTool(renderers=[circ_renderer7]),
)

# use gridplot to show both figures horizontally

handle7 = show(
    gridplot([[sin_fig7, circ_fig7]]),
    notebook_handle=True,
)

# create control handlers

def update_plot7():
    new_data = create_sin_wave_from_state(final_example_state)
    source7.data['sin'] = new_data['sin']
    source7.data['cos'] = new_data['cos']
    push_notebook(handle=handle7)

def amplitude_change_handler7(change):
    final_example_state['amplitude'] = change.new
    update_plot7()

def period_change_handler7(change):
    final_example_state['period'] = change.new
    update_plot7()

def phase_change_handler7(change):
    final_example_state['phase'] = change.new
    update_plot7()

# create the amplitude slider

amplitude_slider7 = widgets.FloatSlider(
    description="amplitude",
    min=0,
    max=5,
    value=final_example_state['amplitude'],
    step=.1,
)
amplitude_slider7.observe(amplitude_change_handler7, names='value')
display(amplitude_slider7)

# create the period slider

period_slider7 = widgets.FloatSlider(
    description="period",
    min=0.1,
    max=4 * math.pi,
    value=final_example_state['period'],
    step=.1,
)
period_slider7.observe(period_change_handler7, names='value')
display(period_slider7)

# create the phase slider

phase_slider7 = widgets.FloatSlider(
    description="phase",
    min=-(2 * math.pi),
    max=(2 * math.pi),
    value=final_example_state['phase'],
    step=.1,
)
phase_slider7.observe(phase_change_handler7, names='value')
display(phase_slider7)